In [1]:
pip install plyer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import sqlite3
import time
import datetime
import threading

# Initialize SQLite database
def init_db():
    try:
        with sqlite3.connect('scheduler.db') as conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS tasks (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    duration REAL NOT NULL,
                    priority TEXT CHECK(priority IN ('high', 'medium', 'low')) NOT NULL,
                    deadline TEXT,
                    scheduled_time TEXT
                )
            ''')
            conn.commit()
    except sqlite3.Error as e:
        print(f"Database error: {e}")

# Add a task to the database
def add_task(name, duration, priority, deadline=None):
    try:
        with sqlite3.connect('scheduler.db') as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO tasks (name, duration, priority, deadline)
                VALUES (?, ?, ?, ?)
            ''', (name, duration, priority, deadline))
            conn.commit()
        print(f"Task '{name}' added successfully!")
    except sqlite3.Error as e:
        print(f"Error adding task: {e}")

# New method to get task input via separate prompts
def get_task_input():
    # Get task name
    while True:
        name = input("Enter task name (e.g., Team Meeting): ").strip()
        if name:
            break
        print("Error: Task name cannot be empty.")

    # Get duration
    while True:
        try:
            duration = float(input("Enter duration in hours (e.g., 1.5): "))
            if duration <= 0:
                print("Error: Duration must be positive.")
            else:
                break
        except ValueError:
            print("Error: Please enter a valid number (e.g., 1.5).")

    # Get priority
    while True:
        priority = input("Enter priority (high, medium, low): ").lower()
        if priority in ['high', 'medium', 'low']:
            break
        print("Error: Priority must be 'high', 'medium', or 'low'.")

    # Get deadline (optional)
    deadline = None
    use_deadline = input("Do you want to set a deadline? (yes/no): ").lower()
    if use_deadline == 'yes':
        while True:
            deadline_input = input("Enter deadline (YYYY-MM-DD HH:MM or 'tomorrow at HH AM/PM'): ")
            try:
                if deadline_input.startswith('tomorrow at'):
                    time_str = deadline_input.replace('tomorrow at', '').strip()
                    tomorrow = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
                    deadline = f"{tomorrow} {time_str}"
                    # Validate deadline format
                    parse_deadline(deadline)
                    break
                else:
                    parse_deadline(deadline_input)
                    deadline = deadline_input
                    break
            except ValueError as e:
                print(f"Error: {e}")

    return {"name": name, "duration": duration, "priority": priority, "deadline": deadline}

# Parse deadline string
def parse_deadline(deadline_str):
    try:
        # Try parsing as "YYYY-MM-DD HH:MM"
        return datetime.datetime.strptime(deadline_str, "%Y-%m-%d %H:%M")
    except ValueError:
        try:
            # Try parsing as "YYYY-MM-DD HH:MM AM/PM"
            return datetime.datetime.strptime(deadline_str, "%Y-%m-%d %I:%M %p")
        except ValueError:
            raise ValueError("Invalid deadline format. Use 'YYYY-MM-DD HH:MM' or 'tomorrow at HH AM/PM'.")

# Fetch all tasks from the database
def get_tasks():
    try:
        with sqlite3.connect('scheduler.db') as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT id, name, duration, priority, deadline FROM tasks")
            tasks = [
                {"id": row[0], "name": row[1], "duration": row[2], "priority": row[3], "deadline": row[4]}
                for row in cursor.fetchall()
            ]
        return tasks
    except sqlite3.Error as e:
        print(f"Error fetching tasks: {e}")
        return []

# Schedule tasks based on priority and work hours
def schedule_tasks(tasks, work_hours={"start": "09:00", "end": "17:00"}):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    start_time = datetime.datetime.strptime(f"{today} {work_hours['start']}", "%Y-%m-%d %H:%M")
    end_time = datetime.datetime.strptime(f"{today} {work_hours['end']}", "%Y-%m-%d %H:%M")

    # Sort tasks by priority (high > medium > low) and deadline
    priority_order = {"high": 1, "medium": 2, "low": 3}
    sorted_tasks = sorted(tasks, key=lambda x: (priority_order[x["priority"]], x["deadline"] or "9999-12-31"))

    schedule = []
    current_time = start_time

    for task in sorted_tasks:
        task_duration = datetime.timedelta(hours=task["duration"])
        if task["deadline"]:
            try:
                deadline_time = parse_deadline(task["deadline"])
                if current_time < deadline_time:
                    scheduled_time = max(current_time, deadline_time - task_duration)
                else:
                    scheduled_time = current_time
            except ValueError as e:
                print(f"Invalid deadline format for task '{task['name']}': {task['deadline']}. Scheduling without deadline.")
                scheduled_time = current_time
        else:
            scheduled_time = current_time

        if scheduled_time + task_duration <= end_time:
            schedule.append({
                "task": task["name"],
                "start_time": scheduled_time.strftime("%Y-%m-%d %H:%M"),
                "duration": task["duration"]
            })
            current_time = scheduled_time + task_duration

            # Update task's scheduled time in the database
            try:
                with sqlite3.connect('scheduler.db') as conn:
                    cursor = conn.cursor()
                    cursor.execute("UPDATE tasks SET scheduled_time = ? WHERE id = ?",
                                 (scheduled_time.strftime("%Y-%m-%d %H:%M"), task["id"]))
                    conn.commit()
            except sqlite3.Error as e:
                print(f"Error updating task '{task['name']}': {e}")

    return schedule

# Simulate reminders with print statements (since plyer notifications don't work in Colab)
def start_reminder_thread():
    def check_reminders():
        while True:
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
            try:
                with sqlite3.connect('scheduler.db') as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT name, scheduled_time FROM tasks WHERE scheduled_time = ?", (current_time,))
                    tasks = cursor.fetchall()

                for task in tasks:
                    print(f"Reminder: Time to start: {task[0]} at {task[1]}")
            except sqlite3.Error as e:
                print(f"Error checking reminders: {e}")
            time.sleep(60)  # Check every minute

    reminder_thread = threading.Thread(target=check_reminders, daemon=True)
    reminder_thread.start()

# Display the schedule
def display_schedule(schedule):
    if not schedule:
        print("No tasks scheduled for today.")
        return
    print("\nToday's Schedule:")
    for item in schedule:
        print(f"{item['start_time']} - {item['task']} ({item['duration']} hours)")

# Main CLI interface
def main():
    init_db()
    start_reminder_thread()

    while True:
        print("\nSmart Routine Scheduler")
        print("1. Add Task")
        print("2. View Schedule")
        print("3. Exit")
        choice = input("Enter choice (1-3): ")

        if choice == "1":
            try:
                task = get_task_input()
                add_task(task["name"], task["duration"], task["priority"], task["deadline"])
            except ValueError as e:
                print(f"Error: {e}")

        elif choice == "2":
            tasks = get_tasks()
            schedule = schedule_tasks(tasks)
            display_schedule(schedule)

        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


Smart Routine Scheduler
1. Add Task
2. View Schedule
3. Exit
Enter choice (1-3): 1
Enter task name (e.g., Team Meeting): workout
Enter duration in hours (e.g., 1.5): 2
Enter priority (high, medium, low): high
Do you want to set a deadline? (yes/no): yes
Enter deadline (YYYY-MM-DD HH:MM or 'tomorrow at HH AM/PM'): 2025-07-20  05:00
Task 'workout' added successfully!

Smart Routine Scheduler
1. Add Task
2. View Schedule
3. Exit
Enter choice (1-3): 2
No tasks scheduled for today.

Smart Routine Scheduler
1. Add Task
2. View Schedule
3. Exit
Enter choice (1-3): 2
No tasks scheduled for today.

Smart Routine Scheduler
1. Add Task
2. View Schedule
3. Exit
Enter choice (1-3): 1
Enter task name (e.g., Team Meeting): walking
Enter duration in hours (e.g., 1.5): 2
Enter priority (high, medium, low): high
Do you want to set a deadline? (yes/no): yes
Enter deadline (YYYY-MM-DD HH:MM or 'tomorrow at HH AM/PM'): 2025-07-17  02:00
Task 'walking' added successfully!

Smart Routine Scheduler
1. Add Ta